In [8]:
import getpass
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

os.environ["LANGCHAIN_TRACING_V2"] = "true"

load_dotenv()
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangChain API key: ")

model = init_chat_model(
    "openai:gpt-4o",
    temperature=0.7
)

In [10]:
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_core.messages import BaseMessage

def get_weather(location: str):
    """Call to get the current weather"""
    if location.lower() in ["sf", "san francisco"]:
        return "It's 60 degrees and foggy"
    else: 
        return "It's 90 degrees and sunny"

tool_node = ToolNode([get_weather]) # Define a tool node that can be called by the LLM

model_with_tools = model.bind_tools([get_weather])

def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls: # Checks if the last message from the LLM included any tool calls.
        return "tools"
    return END

def call_model(state: MessagesState):
    messages = state["messages"]
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}

builder = StateGraph(MessagesState)
builder.add_node("call_model", call_model)
builder.add_node("tools", tool_node)

builder.add_edge(START, "call_model")
builder.add_conditional_edges("call_model", should_continue, ["tools", END])
builder.add_edge("tools", "call_model")

graph = builder.compile()


# Run the graph and capture the result
result = graph.invoke({"messages": [{"role": "user", "content": "what's the weather in sf?"}]})

# Pretty-print the messages like a chat
print("=== Chat Transcript ===")
for msg in result["messages"]:
    if isinstance(msg, BaseMessage):
        role = getattr(msg, "role", msg.__class__.__name__.replace("Message", "").lower())
        content = msg.content.strip() if msg.content else ""
        print(f"\n[{role.upper()}]")
        print(content)
    else:
        print(f"\n[OTHER]\n{msg}")

=== Chat Transcript ===

[HUMAN]
what's the weather in sf?

[AI]


[TOOL]
It's 90 degrees and sunny

[AI]
The weather in San Francisco is currently 90 degrees and sunny.
